In [1]:
from platform import python_version
print(python_version())


# In[3]:

#importing libraries
from astropy.io import fits
from astropy.convolution import convolve, Gaussian2DKernel, Box2DKernel
from astropy.nddata import Cutout2D
from astropy.wcs import WCS
# from reproject import reproject_exact  #a package that can be added to astropy using anaconda or pip (see their docs pg)
# from reproject import reproject_interp

import glob
import matplotlib 
matplotlib.use('Agg') #invokved b/c just plain matplotlib was insufficient
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys


#switches for the three different parts of this code
switch1 = 'on' #convolving images [needed to put it on for switch 3 at min...need to figure out other solution, eh]
switch1b = 'on' #regridding...
switch2 = 'on' #solving equations
switch3 = 'on' #plotting / graphics of solutions


if switch1 == 'on':

    # In[4]:

    # #finding the path to every fits images in a directory
    def im_name_finder(path, file_type):
        #Using glob (it's a unix command similar to ls)
        #WARNING: using recursive=True...depending how many images you use this could be very slow, it's recommended not to have too many subfolders
        #if needed, some example code is commented towards the latter half of this code that could help make an alternative
        all_names = glob.glob(path, recursive=True)

        #IMPORTANT: Using "fit" here because it is inclusive of both fits and FIT...some files end in "FIT" and need to be included
        #using s.lower() include uppercase names
        im_names = [s for s in all_names if 'fit' in s.lower()]

        return im_names


    # In[5]:

    '''now convolve my image with a PSF of the image we're projecting ONTO
    an approx PSF can be found by assuming a 2D Gaussian func with a width (a FWHM) of the diffrac limit
    that is the st dev of the Gaussian is about the st dev is about = lambda/D
    a list of PSFs are found on https://docs.astropy.org/en/stable/convolution/kernels.html

    Notes:
    FIRST: always must convert hdu1_pixtorad to radians! It's inconsistent otherwise, and lambda/D is generally in radians

    what we're using for the gaussian width is the FWHM, not the radius of the first ring of the diffraction pattern,
    so it's 1.2 not 1.22 times lambda/D

    D is 85 cm for spitzer
    D is 2.4 m for hubble
    '''

    def im_conv(low_res_name, D, hdu_pix_torad, hdu_dat, kern, fwhm=None):
        #unfortuantely no good way to find wavelength from header right now. can enter it manually, but I tried to automate it

        #reading in excel file of wavelengths...right now needs to be in same directory as this code
        #first col is a substring of the fits image file name, the second col is the wavelengths in microns
        df = pd.read_excel('../../../imglams.xlsx')
        cols = df.columns
        cols_str = [str(i) for i in df[cols[0]]]
        #some test cases I was using
        
        if kern == 'epsf_fwhm':
            kernel = Gaussian2DKernel(fwhm, y_stddev=fwhm)
        
        #gaussian kernel
        if kern == 'gauss':
            #this finds the loc in the excel file where the image substring matches our image name
            #it then finds the wavelength value corresponding to that loc
            lam =  df.loc[np.where([i in low_res_name for i in cols_str])[0][0]].values[1] #lambda in microns
            
            #finding angular resolution...the FWHM of our Gaussian PSF
            res = 1.2 * lam / D         #resolution in radians
#             print('Angular Res (rads): ', res, ' File Name to Convolve With: ', low_res_name, ' Lambda: ', lam)
            res = res / hdu_pix_torad        #so converting to pixels
#             print('Angular Res (pixels): ', res, ' File Name to Convolve With: ', low_res_name, ' Lambda: ', lam)

            #finding PSF and then calculating the convolution of our image and the PSF of the image we're projecting onto
            kernel = Gaussian2DKernel(res)
        
        #box kernel
        if kern == 'box':
            kernel = Box2DKernel(16.)

        hdu_conv = convolve(hdu_dat, kernel)
        return hdu_conv


    # In[21]:
    
    
    #setting up a new fits file to be saved and viewed in DS9
    #primarily to save the image we reprojected, but can also be used to save the convolved images
    def fits_saver(array, wcs_header, name, save_path):
        '''
        array is a 2d array of data - could be from reprojecting one image onto another or from convolution
        wcs_header is a header containing the wcs coords of the image that we projected onto or of the orig image (if from the convolution)
        name is the path to some image you're using. It will get string split at the / character, and the func only takes the last element of that splitting
        save_path is the folder you want to save to...recommended to also add something to the start of the images names to make it clear what you did to them (e.g. 'Regridded/regrid_')
        '''

        #creating a new file and adding the reprojected array of data as well as the WCS that we projected onto
        hdu_new = fits.PrimaryHDU(array, header=wcs_header)
        hdul = fits.HDUList([hdu_new])

        #saving the file
        new_filename = name.split('/')[-1]  #grabs the file name we were using from before
        hdul.writeto(save_path+new_filename, overwrite=True)

        return (save_path+new_filename)
    
    

    #EX: grabbing all the fits image paths in a directory, so they can be looped through and their data opened
    #set your path to some directory with images (the images can be in subdirectories)
    #using ** will grab all files even in subdirectories...WARNING this will take longer

    # In[28]:
    #this time setting up the file names by hand since I've found that easier...
    #order: halpha or .656 mic, 0.672 mic, 1.26, 1.28, 1.64
    files_units = ['../../../../ngc1333_fits/unregridded/656_image.fits', 
                   '../../../../ngc1333_fits/unregridded/0301_flt.fits', 
                   '../../../../ngc1333_fits/unregridded/0501_flt.fits', 
                   '../../../../ngc1333_fits/126build_shift_2_drz.fits', 
                   '../../../../ngc1333_fits/128build_shift_2_drz.fits', 
                   '../../../../ngc1333_fits/164build_shift_2_drz.fits']
    hdu_list_units = [fits.open(i) for i in files_units]
    files_data = ['../../../../ngc1333_fits/0301_oIreproject2.fits', 
                  '../../../../ngc1333_fits/656_hareproject_shifted_up5_left3.fits', 
                  '../../../../ngc1333_fits/672_sIIreproject.fits', 
                  '../../../../ngc1333_fits/Background_corr/background_corr_126_aligned.fits', 
                  '../../../../ngc1333_fits/Background_corr/background_corr_128_aligned.fits', 
                  '../../../../ngc1333_fits/Background_corr/background_corr_164_aligned.fits']
    hdu_list = [fits.open(i) for i in files_data]

    hdu_pix_list = []
    hdu_pixtorad_list = []
    hdu_fnu_list = []
    hdu_flam_list = []
    hdu_bw_list = []
    hdu_data_list = []
    hdu_header_list = []
    throughput_list = [1., 1., 1., 1., 1., 1.] # [0.242, 1., 0.246, 0.496, 0.521, 0.470] #also has to be done by hand, not in the headers?


    #I'm using count here just to point to specific indices that I've set up...unfortunately some have different headers...
    #the only diff between the if and else cases are the indexing of the hdu's, some need 1 and some need 0
    #I've tried to group it for convience, so the the first two have the same headers, the last 3 have the same headers
    count = 0
    for (hdu_units,hdu_data) in zip(hdu_list_units, hdu_list):
        if count <= 2: # == 0 or count == 2:
            #reading in conversions
            hdu_pix_list.append(hdu_units[0].header['D001SCAL'])  #D001SCAL is the keyword for Hubble images
            hdu_pixtorad_list.append(hdu_pix_list[count] / 206265.)
#             print('D001SCAL: ', hdu_pix_list[count], ' D001SCAL/206265: ', hdu_pixtorad_list[count])
            # hdu_fnu_list.append(hdu_units[1].header['PHOTFNU'])
            hdu_flam_list.append(hdu_units[1].header['PHOTFLAM'])
            hdu_bw_list.append(hdu_units[1].header['PHOTBW'])

            #reading in datafor general use  and header for wcs
            hdu_data_list.append(hdu_data[0].data)
            hdu_header_list.append(hdu_data[0].header)

        else:
            #reading in conversions
            hdu_pix_list.append(hdu_units[0].header['D001SCAL'])  #D001SCAL is the keyword for Hubble images
            hdu_pixtorad_list.append(hdu_pix_list[count] / 206265.)
#             print('D001SCAL: ', hdu_pix_list[count], ' D001SCAL/206265: ', hdu_pixtorad_list[count])
            # hdu_fnu_list.append(hdu_units[0].header['PHOTFNU'])
            hdu_flam_list.append(hdu_units[0].header['PHOTFLAM'])
            hdu_bw_list.append(hdu_units[0].header['PHOTBW'])

            #reading in datafor general use  and header for wcs
            hdu_data_list.append(hdu_data[0].data)
            hdu_header_list.append(hdu_data[0].header)

        count += 1


    #can update later...but basically the sulfur II image header isn't avail...
    #header info taken from https://www.stsci.edu/hst/instrumentation/wfc3/data-analysis/photometric-calibration/uvis-photometric-calibration/quad-filter-photometry
    #update: HAlpha etc uses https://www.stsci.edu/files/live/sites/www/files/home/hst/instrumentation/wfc3/documentation/instrument-science-reports-isrs/_documents/2021/WFC3_ISR_2021-04.pdf
    hdu_flam_list[0] = 1.6600e-17
    hdu_bw_list[0] = 41.77 #from Dan, in A
    hdu_flam_list[2] = 1.3699e-17
    hdu_bw_list[2] = 18.5

    print('loaded data!')



3.8.13


C:\Users\arubi\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


loaded data!


In [2]:
res_str = '656shifted_flam' #used to label what we're saving, usually related to units or whether we're doing a gaussian or box convolution, etc
# resize = 60. #if trying to adjust size of gaussian convolution
D = 2.4 #/ resize #that of Hubble, in m
D *= 1e6 #converting to microns since x m / 1 m * 1E6 microns gets microns, the unit of our wavelength file

#format: conv(data convolving with, Diameter, pix size convolving with, image data to be convolved, convolution method)
#do each with its own psf
# max_fwhm = np.max(fwhm_list)
# fwhm_avg_list = [np.sqrt(max_fwhm**2. - i**2.) for i in fwhm_list]

hdu_conv_list = []

# fwhm_avg_list = [1.1, 1.59, 1.45, 0, 0.95, 1.2] #based on dan
# fwhm_avg_list = [0.7461, 0.65, 0.59, 0, 0.275, 0.6725] #based on astroimagej
# 
fwhm_avg_list = [1.176801567,1.027463529,1.140036453,0,0.968260447,1.062978383] #based on my gaussian2d formula, called sqrtsq_asr2

In [3]:
l = 0 
hdu_conv_list.append(im_conv(files_data[l], D, hdu_pixtorad_list[l], hdu_data_list[l], 'epsf_fwhm', fwhm=fwhm_avg_list[l]))

l = 1
hdu_conv_list.append(im_conv(files_data[l], D, hdu_pixtorad_list[l], hdu_data_list[l], 'epsf_fwhm', fwhm=fwhm_avg_list[l]))

l = 2
hdu_conv_list.append(im_conv(files_data[l], D, hdu_pixtorad_list[l], hdu_data_list[l], 'epsf_fwhm', fwhm=fwhm_avg_list[l]))

l = 3
hdu_conv_list.append(hdu_data_list[l])

l = 4
hdu_conv_list.append(im_conv(files_data[l], D, hdu_pixtorad_list[l], hdu_data_list[l], 'epsf_fwhm', fwhm=fwhm_avg_list[l]))

l = 5
hdu_conv_list.append(im_conv(files_data[l], D, hdu_pixtorad_list[l], hdu_data_list[l], 'epsf_fwhm', fwhm=fwhm_avg_list[l]))

In [4]:
'''
onto converting units
'''
#converting the convolved image to correct units and saving it so we can reproject it
#conversion needed for hubble case since units are not in terms of surface brightness
hdu_conv_scaled_list = []

#for each convolved image, we need to convert from e-/s to flambda units which are in erg/s/cm^2/Angstrom...multiply by bw to get rid of angstrom
#...also images are divided through by throughput, so multiplying by throughput
#and lastly dividing by the arcsec^2 to get the image in surface brightness units, to be used in regridding
for count, i in enumerate(hdu_conv_list):

    #a condition added since the background corrected images have units corrected?
    if  count > 2: #the background subtracted images
        hdu_conv_scaled_list.append(i) #/ hdu_pixtorad_list[count]**2. ) #note if doing regridding should also add in , regrid only takes surface brightness
    elif count <= 2:
        hdu_conv_scaled_list.append(i * hdu_flam_list[count] * hdu_bw_list[count])


# In[6]:
#need a wcs standard for regridding and plots, fits files...
w = WCS(hdu_header_list[-1]) #I picked 0 arbitrarily, it shouldn't really matter
# wcs_header = w.to_header()
wcs_header = hdu_header_list[-1]
wcs_header['HISTORY'] = 'Image was convolved and regridded, units were converted only for 656, 672, and 631 images. Units are all flux in CGS (ergs/s/cm^2).'	#adding history keyword so we know what we did to this image

    #you'll need to set the WCS to be that of the header you're basing this off of...ie the header
file_start = 'conv_checks/conv_byhand_'+res_str+'_'
conv_path_list = [] #list of paths to the convolved images, can be useful...


for count, i in enumerate(hdu_header_list):
    #finding wcs for a given image
    w = WCS(i)
    wcs_header = w.to_header()

    #saving each file to some path, conv_path is the path to that file
    conv_path = fits_saver(hdu_conv_scaled_list[count], wcs_header, files_data[count], file_start)
    conv_path_list.append(conv_path)
